In [36]:
#Get data

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#2020

X2020 = pd.read_csv ('X_2020.csv')
X2020 = X2020.drop(["Unnamed: 0"], axis = 1)
display(X2020)
y2020 = pd.read_csv('y_2020.csv')
y2020 = y2020.drop(["Unnamed: 0"], axis = 1)
y2020 = y2020.values.ravel()
display(y2020)

X_train2020, X_test2020, y_train2020, y_test2020 = train_test_split(X2020, y2020, test_size=0.3)

#1994

X1994 = pd.read_csv ('X_1994.csv')
X1994 = X1994.drop(["Unnamed: 0"], axis = 1)
X1994 = X1994.drop(["capital-loss"], axis = 1)
display(X1994)
y1994 = pd.read_csv('y_1994.csv')
y1994 = y1994.drop(["Unnamed: 0"], axis = 1)
y1994 = y1994.values.ravel()
display(y1994)
X_train1994, X_test1994, y_train1994, y_test1994 = train_test_split(X1994, y1994, test_size=0.3)


,age,fnlwgt,educational-num,capital-gain,hours-per-week,education_cat,workclass_cat,marital-status_cat,occupation_cat,relationship_cat,race_cat,gender_cat,native-country_cat
0,63,166860.0,9,0,40,11,5,2,4,5,4,0,146
1,64,111745.0,9,0,40,11,1,2,10,0,4,1,146
2,54,151825.0,9,0,44,11,2,0,0,1,4,0,146
3,37,174690.0,9,0,21,11,2,4,6,3,4,1,146
4,35,197229.0,11,0,40,8,2,2,8,5,4,0,146
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36249,41,180857.0,9,0,40,11,2,2,4,0,1,1,146
36250,58,117245.0,9,0,55,11,2,2,10,0,3,1,146
36251,31,126498.0,9,0,40,11,2,4,6,2,3,0,146
36252,57,72015.0,13,0,50,9,2,0,10,4,3,0,146


array(['<=90k', '<=90k', '<=90k', ..., '<=90k', '>90k', '<=90k'],
      dtype=object)

,age,fnlwgt,educational-num,capital-gain,hours-per-week,education_cat,workclass_cat,marital-status_cat,occupation_cat,relationship_cat,race_cat,gender_cat,native-country_cat
0,25,226802.0,7,0,40,1,2,4,6,3,2,1,38
1,38,89814.0,9,0,50,11,2,2,4,0,4,1,38
2,28,336951.0,12,0,40,7,1,2,10,0,4,1,38
3,44,160323.0,10,7688,40,15,2,2,6,0,2,1,38
4,34,198693.0,6,0,30,0,2,4,7,1,4,1,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45217,27,257302.0,12,0,38,7,2,2,12,5,4,0,38
45218,40,154374.0,9,0,40,11,2,2,6,0,4,1,38
45219,58,151910.0,9,0,40,11,2,6,0,4,4,0,38
45220,22,201490.0,9,0,20,11,2,4,0,3,4,1,38


array(['<=50K', '<=50K', '>50K', ..., '<=50K', '<=50K', '>50K'],
      dtype=object)

In [37]:
index = ['LR 2020', 'LR 1994', 'RF 2020', 'RF 1994', 'BT2020', 'BT1994', 'CB 2020', 'CB 1994', 'XB 2020', 'XB 1994']
featImportances = pd.DataFrame(index = X_train2020.columns)
# featImportances.index = ['LR 2020', 'LR 1994', 'RF 2020', 'RF 1994', 'BT2020', 'BT1994', 'CB 2020', 'CB 1994', 'XB 2020', 'XB 1994']
# featImportances.columns = X_train2020.columns

In [38]:
#Random Forest
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier

#2020
params = {'n_estimators' : 1200,'max_depth' : 15,'min_samples_split' : 5,'min_samples_leaf' : 5}
model = RandomForestClassifier()
model.set_params(**params)
model.fit(X_train2020, y_train2020)
preds = model.predict_proba(X_test2020)
print("2020 RF ROC AUC: ", roc_auc_score(y_test2020, preds[:,1]))
feats = {}
for feature, importance in zip(X_train2020.columns, model.feature_importances_): 
  feats[feature] = importance
featImportances = pd.DataFrame.from_dict(feats, orient = 'index').rename(columns = {0: 'RF 2020'})

#1994  
params = {'n_estimators' : 1200,'max_depth' : 15,'min_samples_split' : 5,'min_samples_leaf' : 1}
model = RandomForestClassifier()
model.set_params(**params)
model.fit(X_train1994, y_train1994)
preds = model.predict_proba(X_test1994)
print("1994 RF ROC AUC: ", roc_auc_score(y_test1994, preds[:,1]))


2020 RF ROC AUC:  0.8576261477555864
1994 RF ROC AUC:  0.9139762965709435


In [39]:
for feature, importance in zip(X_train2020.columns, model.feature_importances_): 
  feats[feature] = importance
featImportanceTemp = pd.DataFrame.from_dict(feats, orient = 'index').rename(columns = {0: 'RF 1994'})
featImportances['RF 1994'] = featImportanceTemp['RF 1994']
display(featImportances)

,RF 2020,RF 1994
age,0.124632,0.106026
fnlwgt,0.093670,0.065698
educational-num,0.228927,0.138990
capital-gain,0.069628,0.180708
hours-per-week,0.115281,0.068443
education_cat,0.040551,0.043016
workclass_cat,0.037217,0.026114
marital-status_cat,0.043334,0.109518
occupation_cat,0.072855,0.051038
relationship_cat,0.110814,0.170808


In [40]:
#Boosted Trees
from sklearn.ensemble import GradientBoostingClassifier

#2020

params = {'subsample': 1.0, 'n_estimators': 100, 'min_samples_split': 100, 'min_samples_leaf': 2, 
          'max_depth': 7, 'loss': 'exponential', 'learning_rate': 0.1}
model = GradientBoostingClassifier()
model.set_params(**params)
model.fit(X_train2020, y_train2020)
preds = model.predict_proba(X_test2020)
print("2020 BT ROC AUC: ", roc_auc_score(y_test2020, preds[:,1]))
for feature, importance in zip(X_train2020.columns, model.feature_importances_): 
  feats[feature] = importance
featImportanceTemp = pd.DataFrame.from_dict(feats, orient = 'index').rename(columns = {0: 'BT 2020'})
featImportances['BT 2020'] = featImportanceTemp['BT 2020']
# display(featImportances)


#1994
params = {'subsample': 1.0, 'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 5,
          'max_depth': 7, 'loss': 'exponential', 'learning_rate': 0.1}
model = GradientBoostingClassifier()
model.set_params(**params)
model.fit(X_train1994, y_train1994)
preds = model.predict_proba(X_test1994)
print("1994 BT ROC AUC: ", roc_auc_score(y_test1994, preds[:,1]))
for feature, importance in zip(X_train1994.columns, model.feature_importances_): 
  feats[feature] = importance
featImportanceTemp = pd.DataFrame.from_dict(feats, orient = 'index').rename(columns = {0: 'BT 1994'})
featImportances['BT 1994'] = featImportanceTemp['BT 1994']
display(featImportances)

2020 BT ROC AUC:  0.8629459802584177
1994 BT ROC AUC:  0.9181408911465821


,RF 2020,RF 1994,BT 2020,BT 1994
age,0.124632,0.106026,0.093990,0.079791
fnlwgt,0.093670,0.065698,0.090204,0.099382
educational-num,0.228927,0.138990,0.293485,0.178946
capital-gain,0.069628,0.180708,0.064308,0.188092
hours-per-week,0.115281,0.068443,0.115797,0.048728
education_cat,0.040551,0.043016,0.019908,0.011401
workclass_cat,0.037217,0.026114,0.041369,0.018480
marital-status_cat,0.043334,0.109518,0.010515,0.011760
occupation_cat,0.072855,0.051038,0.074272,0.044618
relationship_cat,0.110814,0.170808,0.157266,0.299070


In [41]:
#CatBoost
!pip install catboost
from catboost import CatBoostClassifier

#2020
params = {'od_wait': 300, 'iterations': 1000, 'l2_leaf_reg': 0.1}
model = CatBoostClassifier()
model.set_params(**params)
model.fit(X_train2020, y_train2020)
preds = model.predict_proba(X_test2020)
print("2020 CB ROC AUC: ", roc_auc_score(y_test2020, preds[:,1]))
for feature, importance in zip(X_train2020.columns, model.feature_importances_): 
  feats[feature] = importance
featImportanceTemp = pd.DataFrame.from_dict(feats, orient = 'index').rename(columns = {0: 'CB 2020'})
featImportances['CB 2020'] = featImportanceTemp['CB 2020']


#1994
params = {'od_wait': 300, 'iterations': 1000, 'l2_leaf_reg': 0.1}
model = CatBoostClassifier()
model.set_params(**params)
model.fit(X_train1994, y_train1994)
preds = model.predict_proba(X_test1994)
print("1994 CB ROC AUC: ", roc_auc_score(y_test1994, preds[:,1]))
for feature, importance in zip(X_train1994.columns, model.feature_importances_): 
  feats[feature] = importance
featImportanceTemp = pd.DataFrame.from_dict(feats, orient = 'index').rename(columns = {0: 'CB 1994'})
featImportances['CB 1994'] = featImportanceTemp['CB 1994']
display(featImportances)

0:	learn: 0.6702661	total: 13.4ms	remaining: 13.4s
1:	learn: 0.6488494	total: 27.1ms	remaining: 13.5s
2:	learn: 0.6276114	total: 39.5ms	remaining: 13.1s
3:	learn: 0.6069892	total: 51.9ms	remaining: 12.9s
4:	learn: 0.5886991	total: 64.5ms	remaining: 12.8s
5:	learn: 0.5736263	total: 77.6ms	remaining: 12.9s
6:	learn: 0.5575750	total: 90.1ms	remaining: 12.8s
7:	learn: 0.5449606	total: 103ms	remaining: 12.7s
8:	learn: 0.5330533	total: 115ms	remaining: 12.7s
9:	learn: 0.5221302	total: 128ms	remaining: 12.6s
10:	learn: 0.5123891	total: 141ms	remaining: 12.6s
11:	learn: 0.5020551	total: 154ms	remaining: 12.7s
12:	learn: 0.4931460	total: 166ms	remaining: 12.6s
13:	learn: 0.4851577	total: 179ms	remaining: 12.6s
14:	learn: 0.4785311	total: 192ms	remaining: 12.6s
15:	learn: 0.4703007	total: 204ms	remaining: 12.6s
16:	learn: 0.4632466	total: 221ms	remaining: 12.8s
17:	learn: 0.4569160	total: 235ms	remaining: 12.8s
18:	learn: 0.4514867	total: 247ms	remaining: 12.8s
19:	learn: 0.4460807	total: 260ms	

,RF 2020,RF 1994,BT 2020,BT 1994,CB 2020,CB 1994
age,0.124632,0.106026,0.093990,0.079791,14.642551,12.530675
fnlwgt,0.093670,0.065698,0.090204,0.099382,8.963143,5.638790
educational-num,0.228927,0.138990,0.293485,0.178946,14.439790,9.279827
capital-gain,0.069628,0.180708,0.064308,0.188092,3.099395,19.645403
hours-per-week,0.115281,0.068443,0.115797,0.048728,12.894946,9.253127
education_cat,0.040551,0.043016,0.019908,0.011401,3.885450,4.759400
workclass_cat,0.037217,0.026114,0.041369,0.018480,5.857212,3.240370
marital-status_cat,0.043334,0.109518,0.010515,0.011760,3.435743,5.917049
occupation_cat,0.072855,0.051038,0.074272,0.044618,13.849365,8.047676
relationship_cat,0.110814,0.170808,0.157266,0.299070,7.418266,15.539541


In [42]:
display(featImportances)

,RF 2020,RF 1994,BT 2020,BT 1994,CB 2020,CB 1994
age,0.124632,0.106026,0.093990,0.079791,14.642551,12.530675
fnlwgt,0.093670,0.065698,0.090204,0.099382,8.963143,5.638790
educational-num,0.228927,0.138990,0.293485,0.178946,14.439790,9.279827
capital-gain,0.069628,0.180708,0.064308,0.188092,3.099395,19.645403
hours-per-week,0.115281,0.068443,0.115797,0.048728,12.894946,9.253127
education_cat,0.040551,0.043016,0.019908,0.011401,3.885450,4.759400
workclass_cat,0.037217,0.026114,0.041369,0.018480,5.857212,3.240370
marital-status_cat,0.043334,0.109518,0.010515,0.011760,3.435743,5.917049
occupation_cat,0.072855,0.051038,0.074272,0.044618,13.849365,8.047676
relationship_cat,0.110814,0.170808,0.157266,0.299070,7.418266,15.539541


In [43]:
#XGBoost
from xgboost import XGBClassifier

#2020
params = {'max_depth': 4, 'min_split_loss': 1, 'n_estimators': 500, 'objective': 'binary:logistic', 'subsample': 1}
model = XGBClassifier()
model.set_params(**params)
model.fit(X_train2020, y_train2020)
preds = model.predict_proba(X_test2020)
print("2020 XB ROC AUC: ", roc_auc_score(y_test2020, preds[:,1]))
for feature, importance in zip(X_train2020.columns, model.feature_importances_): 
  feats[feature] = importance
featImportanceTemp = pd.DataFrame.from_dict(feats, orient = 'index').rename(columns = {0: 'XB 2020'})
featImportances['XB 2020'] = featImportanceTemp['XB 2020']

#1994

params = {'max_depth': 4, 'min_split_loss': 0, 'n_estimators': 500, 'objective': 'binary:logistic', 'subsample': 1}
model = XGBClassifier()
model.set_params(**params)
model.fit(X_train1994, y_train1994)
preds = model.predict_proba(X_test1994)
print("1994 XB ROC AUC: ", roc_auc_score(y_test1994, preds[:,1]))
for feature, importance in zip(X_train1994.columns, model.feature_importances_): 
  feats[feature] = importance
featImportanceTemp = pd.DataFrame.from_dict(feats, orient = 'index').rename(columns = {0: 'XB 1994'})
featImportances['XB 1994'] = featImportanceTemp['XB 1994']
display(featImportances)

2020 XB ROC AUC:  0.8644429801833255
1994 XB ROC AUC:  0.9203696886476902


,RF 2020,RF 1994,BT 2020,BT 1994,CB 2020,CB 1994,XB 2020,XB 1994
age,0.124632,0.106026,0.093990,0.079791,14.642551,12.530675,0.058601,0.040994
fnlwgt,0.093670,0.065698,0.090204,0.099382,8.963143,5.638790,0.020032,0.010482
educational-num,0.228927,0.138990,0.293485,0.178946,14.439790,9.279827,0.289277,0.156392
capital-gain,0.069628,0.180708,0.064308,0.188092,3.099395,19.645403,0.070666,0.114153
hours-per-week,0.115281,0.068443,0.115797,0.048728,12.894946,9.253127,0.072723,0.031182
education_cat,0.040551,0.043016,0.019908,0.011401,3.885450,4.759400,0.020436,0.009580
workclass_cat,0.037217,0.026114,0.041369,0.018480,5.857212,3.240370,0.043921,0.015325
marital-status_cat,0.043334,0.109518,0.010515,0.011760,3.435743,5.917049,0.031118,0.118780
occupation_cat,0.072855,0.051038,0.074272,0.044618,13.849365,8.047676,0.070139,0.031866
relationship_cat,0.110814,0.170808,0.157266,0.299070,7.418266,15.539541,0.210205,0.419732


In [44]:
from google.colab import drive
drive.mount('drive')
featImportances.to_csv('featImportances.csv')

Mounted at drive


In [55]:
print(featImportances['RF 2020'].sum())
print(featImportances['CB 2020'].sum())
print(featImportances['CB 1994'].sum())

0.9999999999999999
1.0
0.9999999999999999


In [52]:
featImportances['CB 2020'] =featImportances['CB 2020']
featImportances['CB 1994'] =featImportances['CB 1994']*100
display(featImportances)

,Unnamed: 0,RF 2020,RF 1994,BT 2020,BT 1994,CB 2020,CB 1994,XB 2020,XB 1994
0,age,0.124632,0.106026,0.093990,0.079791,0.146426,0.125307,0.058601,0.040994
1,fnlwgt,0.093670,0.065698,0.090204,0.099382,0.089631,0.056388,0.020032,0.010482
2,educational-num,0.228927,0.138990,0.293485,0.178946,0.144398,0.092798,0.289277,0.156392
3,capital-gain,0.069628,0.180708,0.064308,0.188092,0.030994,0.196454,0.070666,0.114153
4,hours-per-week,0.115281,0.068443,0.115797,0.048728,0.128949,0.092531,0.072723,0.031182
5,education_cat,0.040551,0.043016,0.019908,0.011401,0.038855,0.047594,0.020436,0.009580
6,workclass_cat,0.037217,0.026114,0.041369,0.018480,0.058572,0.032404,0.043921,0.015325
7,marital-status_cat,0.043334,0.109518,0.010515,0.011760,0.034357,0.059170,0.031118,0.118780
8,occupation_cat,0.072855,0.051038,0.074272,0.044618,0.138494,0.080477,0.070139,0.031866
9,relationship_cat,0.110814,0.170808,0.157266,0.299070,0.074183,0.155395,0.210205,0.419732


In [59]:
featImportances = featImportances.sort_values
ft1994 = pd.DataFrame(index = X_train2020.columns)
ft1994['RF 1994'] = featImportances['RF 1994'].to_numpy()
ft1994['BT 1994'] = featImportances['BT 1994'].to_numpy()
ft1994['CB 1994'] = featImportances['CB 1994'].to_numpy()
ft1994['XB 1994'] = featImportances['XB 1994'].to_numpy()
display(ft1994)

ft2020 = pd.DataFrame(index = X_train2020.columns)
ft2020['RF 2020'] = featImportances['RF 2020'].to_numpy()
ft2020['BT 2020'] = featImportances['BT 2020'].to_numpy()
ft2020['CB 2020'] = featImportances['CB 2020'].to_numpy()
ft2020['XB 2020'] = featImportances['XB 2020'].to_numpy()
display(ft2020)


,RF 1994,BT 1994,CB 1994,XB 1994
age,0.106026,0.079791,0.125307,0.040994
fnlwgt,0.065698,0.099382,0.056388,0.010482
educational-num,0.138990,0.178946,0.092798,0.156392
capital-gain,0.180708,0.188092,0.196454,0.114153
hours-per-week,0.068443,0.048728,0.092531,0.031182
education_cat,0.043016,0.011401,0.047594,0.009580
workclass_cat,0.026114,0.018480,0.032404,0.015325
marital-status_cat,0.109518,0.011760,0.059170,0.118780
occupation_cat,0.051038,0.044618,0.080477,0.031866
relationship_cat,0.170808,0.299070,0.155395,0.419732


,RF 2020,BT 2020,CB 2020,XB 2020
age,0.124632,0.093990,0.146426,0.058601
fnlwgt,0.093670,0.090204,0.089631,0.020032
educational-num,0.228927,0.293485,0.144398,0.289277
capital-gain,0.069628,0.064308,0.030994,0.070666
hours-per-week,0.115281,0.115797,0.128949,0.072723
education_cat,0.040551,0.019908,0.038855,0.020436
workclass_cat,0.037217,0.041369,0.058572,0.043921
marital-status_cat,0.043334,0.010515,0.034357,0.031118
occupation_cat,0.072855,0.074272,0.138494,0.070139
relationship_cat,0.110814,0.157266,0.074183,0.210205


In [ ]:
feat_importances = pd.Series(boost.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.title("Random Forest Feature Importance")

In [62]:
fiavg =  pd.read_csv ('fiavg.csv')
display(fiavg)
fiavg = fiavg.sort_values(by='1994 avg', ascending =False)
display(fiavg)

,Model,1994 avg,2020 avg
0,age,0.088029,0.105912
1,fnlwgt,0.057987,0.073384
2,educational-num,0.141782,0.239022
3,capital-gain,0.169852,0.058899
4,hours-per-week,0.060221,0.108188
5,education_cat,0.027898,0.029937
6,workclass_cat,0.023081,0.045270
7,marital-status_cat,0.074807,0.029831
8,occupation_cat,0.052000,0.088940
9,relationship_cat,0.261251,0.138117


,Model,1994 avg,2020 avg
9,relationship_cat,0.261251,0.138117
3,capital-gain,0.169852,0.058899
2,educational-num,0.141782,0.239022
0,age,0.088029,0.105912
7,marital-status_cat,0.074807,0.029831
4,hours-per-week,0.060221,0.108188
1,fnlwgt,0.057987,0.073384
8,occupation_cat,0.052000,0.088940
5,education_cat,0.027898,0.029937
6,workclass_cat,0.023081,0.045270
